Reutilizo una funcion del primer proyecto: CalcularNulos

In [1]:
import pandas as pd

def CalcularNulos(df):
    for columna in df:
        nulos = 100*(df[columna].isnull().sum()/df.shape[0])
        porcentaje = "{:.1f}".format(nulos)
        print("el porcentaje de datos nulos en la columna ",df[columna].name," es del ",porcentaje,"%")

    print('-------------------------------------------------------------------------------------------')

def ConteoColumnas(df):
    for columna in df.columns:
        print(df[columna].value_counts())
        print('----------------------------------------------------------------------------------')

#########################################################################################################
___

**ETL Hechos**

In [2]:
archivo = pd.ExcelFile('../archivos/originales/homicidios.xlsx')
df_hechos = pd.read_excel(archivo,sheet_name="HECHOS")
df_victimas = pd.read_excel(archivo,sheet_name="VICTIMAS")

aca reemplazo los valores Sin Datos, a None para hacer un conteo de los nulos que hay en el dataframe

In [3]:
df_hechos = df_hechos.replace('SD',None)

In [4]:
CalcularNulos(df_hechos)

el porcentaje de datos nulos en la columna  ID  es del  0.0 %
el porcentaje de datos nulos en la columna  N_VICTIMAS  es del  0.0 %
el porcentaje de datos nulos en la columna  FECHA  es del  0.0 %
el porcentaje de datos nulos en la columna  AAAA  es del  0.0 %
el porcentaje de datos nulos en la columna  MM  es del  0.0 %
el porcentaje de datos nulos en la columna  DD  es del  0.0 %
el porcentaje de datos nulos en la columna  HORA  es del  0.1 %
el porcentaje de datos nulos en la columna  HH  es del  0.1 %
el porcentaje de datos nulos en la columna  LUGAR_DEL_HECHO  es del  0.1 %
el porcentaje de datos nulos en la columna  TIPO_DE_CALLE  es del  0.0 %
el porcentaje de datos nulos en la columna  Calle  es del  0.1 %
el porcentaje de datos nulos en la columna  Altura  es del  81.5 %
el porcentaje de datos nulos en la columna  Cruce  es del  24.6 %
el porcentaje de datos nulos en la columna  Dirección Normalizada  es del  1.1 %
el porcentaje de datos nulos en la columna  COMUNA  es del  0.

comienzo verificando la columna HH\
*No se puede imputar valor faltante, aun asi contiene otros datos relevantes, como el tipo de calle, la fecha, el lugar de acontecimiento y las victimas, no se borra*

In [5]:
hh = df_hechos[df_hechos['HH'].isnull()]
hh

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
518,2019-0103,1,2019-12-18,2019,12,18,None,None,"PAZ, GRAL. AV. Y GRIVEO",GRAL PAZ,...,NaN,GRIVEO,"PAZ, GRAL. AV. y GRIVEO",11,Point (94643.11254058 103831.57115061),-58.52169422,-34.59471640,MOTO-MOTO,MOTO,MOTO


+ Altura se elimina, tengo pos x(longitud) y pos y(latitud) como coordenadas para utilizar en un mapa
+ Cruce lo convierto a 1 y 0, con el fin de identificar cuando el choque ocurrio en una esquina o no.
+ lugar del hecho y direccion normalizada son columnas relacionadas, voy a eliminar lugar del hecho tras copiar el dato faltante de direccion normalizada
+ fecha contiene los datos de las columnas AAAA, MM, DD. Eliminare solo la columna DD, ya que AAAA y MM me permitiran agrupar mas rapido
+ utilizaremos unicamente el estandar de HH, para indicar ventanas de tiempo y eliminaremos la columna HORA


In [16]:
df_hechos_clean = df_hechos.drop(['Altura','LUGAR_DEL_HECHO','DD','HORA'], axis=1)

df_hechos_clean['Cruce'] = df_hechos_clean['Cruce'].notna().astype(int)

In [17]:
df_hechos_clean.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,HH,TIPO_DE_CALLE,Calle,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,4,AVENIDA,PIEDRA BUENA AV.,1,"PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,1,GRAL PAZ,"PAZ, GRAL. AV.",1,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,7,AVENIDA,ENTRE RIOS AV.,0,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,0,AVENIDA,LARRAZABAL AV.,1,"LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,None
4,2016-0005,1,2016-01-21,2016,1,5,AVENIDA,SAN JUAN AV.,1,"SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


Chequeo si puedo imputar el valor faltante de calle con direccion normalizada

In [18]:
calles_nulas = df_hechos[df_hechos['Calle'].isnull()]
calles_nulas.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
119,2016-0151,1,2016-11-18,2016,11,18,20:35:00,20,None,CALLE,...,NaN,NaN,NaN,0,Point (. .),.,.,PEATON-SD,PEATON,None


No encontre nada en esta fila que me pueda ser de utilidad, elimino la fila donde aparece el valor nulo y la columna direccion normalizada

In [19]:
df_hechos_clean = df_hechos_clean.dropna(subset=['Calle'])
df_hechos_clean = df_hechos_clean.drop('Dirección Normalizada', axis=1)

filtro las columnas VICTIMA y ACUSADO para ver si puedo imputar los datos faltantes con la columna PARTICIPANTES

In [20]:
victima = df_hechos_clean[df_hechos_clean['VICTIMA'].isnull()]
victima.shape
victima.head(9)

,ID,N_VICTIMAS,FECHA,AAAA,MM,HH,TIPO_DE_CALLE,Calle,Cruce,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
35,2016-0049,1,2016-04-17,2016,4,0,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,0,4,Point (. .),-58.37714647568196,-34.63657525428238,SD-SD,None,None
76,2016-0101,1,2016-08-07,2016,8,19,GRAL PAZ,"PAZ, GRAL. AV.",1,9,Point (93946.81673714 97681.68907836),-58.52932872,-34.65014993,SD-SD,None,None
88,2016-0115,1,2016-09-02,2016,9,11,CALLE,ARIAS,1,12,Point (97608.87024259 108884.67029908),-58.48935041,-34.54917673,SD-CARGAS,None,CARGAS
92,2016-0119,1,2016-09-04,2016,9,6,AVENIDA,"LARRALDE, CRISOLOGO AV.",1,12,Point (96944.69676081 107291.52292157),-58.49659234,-34.56353635,SD-SD,None,None
139,2016-0174,1,2016-12-27,2016,12,0,AUTOPISTA,AUTOPISTA 25 DE MAYO,0,0,Point (. .),.,.,SD-SD,None,None
164,2017-0029,1,2017-03-07,2017,3,2,CALLE,MOZART,1,9,Point (98009.35575364 97985.81952007),-58.48501341,-34.64742455,SD-AUTO,None,AUTO
201,2017-0074,1,2017-06-04,2017,6,12,CALLE,CASCO,0,9,Point (94281.31244858 99043.29027489),-58.52567073,-34.63787791,SD-SD,None,None
214,2017-0089,1,2017-07-13,2017,7,12,CALLE,"LEGUIZAMON, MARTINIANO",0,9,Point (95246.78684010 97935.32361317),-58.51514709,-34.64787051,SD-AUTO,None,AUTO
269,2017-0155,1,2017-12-12,2017,12,21,AVENIDA,RIVADAVIA AV.,0,7,Point (100674.98109200 100299.64949548),-58.45593915,-34.62656868,SD-MOTO,None,MOTO


In [21]:
acusado = df_hechos_clean[df_hechos_clean['ACUSADO'].isnull()]
acusado.shape
acusado.head(9)

,ID,N_VICTIMAS,FECHA,AAAA,MM,HH,TIPO_DE_CALLE,Calle,Cruce,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
3,2016-0004,1,2016-01-10,2016,1,0,AVENIDA,LARRAZABAL AV.,1,8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,None
32,2016-0045,1,2016-04-11,2016,4,10,GRAL PAZ,"PAZ, GRAL. AV.",1,13,Point (99620.34936816 110483.29286598),-58.46743471,-34.53476874,MOTO-SD,MOTO,None
35,2016-0049,1,2016-04-17,2016,4,0,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,0,4,Point (. .),-58.37714647568196,-34.63657525428238,SD-SD,None,None
38,2016-0052,1,2016-04-20,2016,4,20,AUTOPISTA,"LUGONES, LEOPOLDO AV.",0,13,Point (. .),.,.,MOTO-SD,MOTO,None
57,2016-0080,1,2016-06-18,2016,6,6,GRAL PAZ,"PAZ, GRAL. AV.",1,9,Point (93950.50587297 98883.37644525),-58.52927982,-34.63931752,AUTO-SD,AUTO,None
76,2016-0101,1,2016-08-07,2016,8,19,GRAL PAZ,"PAZ, GRAL. AV.",1,9,Point (93946.81673714 97681.68907836),-58.52932872,-34.65014993,SD-SD,None,None
83,2016-0110,1,2016-08-24,2016,8,20,AVENIDA,"ALBERDI, JUAN BAUTISTA AV.",1,9,Point (95506.35959227 96696.89655626),-58.51232237,-34.65903530,PEATON-SD,PEATON,None
92,2016-0119,1,2016-09-04,2016,9,6,AVENIDA,"LARRALDE, CRISOLOGO AV.",1,12,Point (96944.69676081 107291.52292157),-58.49659234,-34.56353635,SD-SD,None,None
105,2016-0135,1,2016-10-24,2016,10,5,AVENIDA,"FIGUEROA ALCORTA, PRES. AV.",1,14,Point (103734.76257993 107235.78328415),-58.42260225,-34.56403694,PEATON-SD,PEATON,None


Si bien no puedo imputar los datos faltantes conservo las filas ya que tienen ubicaciones y ventanas de tiempo necesarias para analizar\
*Lo que si elimino es la columna Participantes

In [22]:
df_hechos_clean = df_hechos_clean.drop(columns=['PARTICIPANTES'],axis=1)

mientras chequeaba las columnas anteriores, verifique que hay valores faltantes en pos x y pos y

In [23]:
nul_x = df_hechos_clean[df_hechos_clean['pos x']=='.']
nul_y = df_hechos_clean[df_hechos_clean['pos y']=='.']

In [24]:
nul_x.shape
nul_y.shape
#comparten las mismas filas
nul_x.head(11)

,ID,N_VICTIMAS,FECHA,AAAA,MM,HH,TIPO_DE_CALLE,Calle,Cruce,COMUNA,XY (CABA),pos x,pos y,VICTIMA,ACUSADO
38,2016-0052,1,2016-04-20,2016,4,20,AUTOPISTA,"LUGONES, LEOPOLDO AV.",0,13,Point (. .),.,.,MOTO,None
106,2016-0136,1,2016-10-25,2016,10,0,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,0,4,Point (. .),.,.,MOTO,CARGAS
139,2016-0174,1,2016-12-27,2016,12,0,AUTOPISTA,AUTOPISTA 25 DE MAYO,0,0,Point (. .),.,.,None,None
176,2017-0042,1,2017-04-10,2017,4,9,GRAL PAZ,"LUGONES, LEOPOLDO AV.",0,14,Point (. .),.,.,MOTO,CARGAS
180,2017-0050,2,2017-04-28,2017,4,11,AUTOPISTA,AUTOPISTA PERITO MORENO,0,9,Point (. .),.,.,MOTO,CARGAS
181,2017-0051,1,2017-05-01,2017,5,3,AUTOPISTA,AUTOPISTA DELLEPIANE LUIS TTE. GRAL.,0,7,Point (. .),.,.,AUTO,AUTO
256,2017-0140,1,2017-11-19,2017,11,23,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,0,4,Point (. .),.,.,MOTO,PASAJEROS
313,2018-0039,1,2018-04-21,2018,4,22,AUTOPISTA,"LUGONES, LEOPOLDO AV.",0,14,Point (. .),.,.,PEATON,AUTO
546,2020-0026,1,2020-05-17,2020,5,6,AUTOPISTA,"LUGONES, LEOPOLDO AV.",0,14,Point (. .),.,.,MOTO,OBJETO FIJO
559,2020-0039,1,2020-09-01,2020,9,19,CALLE,MURGUIONDO,0,9,Point (. .),.,.,PEATON,CARGAS


No se pueden imputar, se matienen las filas pero se borra la columna XY (CABA)

In [25]:
df_hechos_clean = df_hechos_clean.drop('XY (CABA)',axis=1)

In [26]:
CalcularNulos(df_hechos_clean)

el porcentaje de datos nulos en la columna  ID  es del  0.0 %
el porcentaje de datos nulos en la columna  N_VICTIMAS  es del  0.0 %
el porcentaje de datos nulos en la columna  FECHA  es del  0.0 %
el porcentaje de datos nulos en la columna  AAAA  es del  0.0 %
el porcentaje de datos nulos en la columna  MM  es del  0.0 %
el porcentaje de datos nulos en la columna  HH  es del  0.1 %
el porcentaje de datos nulos en la columna  TIPO_DE_CALLE  es del  0.0 %
el porcentaje de datos nulos en la columna  Calle  es del  0.0 %
el porcentaje de datos nulos en la columna  Cruce  es del  0.0 %
el porcentaje de datos nulos en la columna  COMUNA  es del  0.0 %
el porcentaje de datos nulos en la columna  pos x  es del  0.0 %
el porcentaje de datos nulos en la columna  pos y  es del  0.0 %
el porcentaje de datos nulos en la columna  VICTIMA  es del  1.3 %
el porcentaje de datos nulos en la columna  ACUSADO  es del  3.2 %
----------------------------------------------------------------------------------

In [28]:
df_hechos_clean = df_hechos_clean.rename(columns={'pos x':'Longitud','pos y':'Latitud'})

+ el primer REGEX lo use para filtrar las filas que contenian caracteres especiales
+ el segundo para poder buscar los '?' signos de pregunta que estaban entre vocales (por ejemplo: NU?EZ), y asi con la funcion str.replace(), reemplazar estos por una 'Ñ'

In [29]:
car_especiales = r'[^\w\s.,]'
entre_vocales = r'(?<=[aeiouáéíóúüAEIOUÁÉÍÓÚÜ])\?(?=[aeiouáéíóúüAEIOUÁÉÍÓÚÜ])'

In [31]:
df_hechos_clean['Calle'] = df_hechos_clean['Calle'].str.replace(entre_vocales,'Ñ',regex=True)

In [32]:
faltantes = df_hechos_clean[df_hechos_clean['Calle'].str.contains(car_especiales)]
faltantes

,ID,N_VICTIMAS,FECHA,AAAA,MM,HH,TIPO_DE_CALLE,Calle,Cruce,COMUNA,Longitud,Latitud,VICTIMA,ACUSADO
106,2016-0136,1,2016-10-25,2016,10,0,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,0,4,.,.,MOTO,CARGAS
370,2018-0096,1,2018-08-31,2018,8,22,AUTOPISTA,AUTOPISTA PRESIDENTE H?CTOR J. C?MPORA,1,8,-58.44483258,-34.67152226,PEATON,AUTO
621,2021-0023,1,2021-03-01,2021,3,9,AUTOPISTA,AUTOPISTA BUENOS AIRES - LA PLATA,0,4,.,.,MOTO,CARGAS


Solo reemplazo la fila donde se encuentra la calle con los signos de pregunta: 'AUTOPISTA PRESIDENTE H?CTOR J. C?MPORA'

In [33]:
df_hechos_clean['Calle'] = df_hechos_clean['Calle'].str.replace('AUTOPISTA PRESIDENTE H?CTOR J. C?MPORA','AUTOPISTA PRESIDENTE HECTOR J. CAMPORA')

In [35]:
df_hechos_clean.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,HH,TIPO_DE_CALLE,Calle,Cruce,COMUNA,Longitud,Latitud,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,4,AVENIDA,PIEDRA BUENA AV.,1,8,-58.47533969,-34.68757022,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,1,GRAL PAZ,"PAZ, GRAL. AV.",1,9,-58.50877521,-34.66977709,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,7,AVENIDA,ENTRE RIOS AV.,0,1,-58.39040293,-34.63189362,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,0,AVENIDA,LARRAZABAL AV.,1,8,-58.46503904,-34.68092974,MOTO,None
4,2016-0005,1,2016-01-21,2016,1,5,AVENIDA,SAN JUAN AV.,1,1,-58.38718297,-34.62246630,MOTO,PASAJEROS


In [36]:
df_hechos_clean.columns = [col.capitalize() for col in df_hechos_clean.columns]

In [37]:
df_hechos_clean.head()

,Id,N_victimas,Fecha,Aaaa,Mm,Hh,Tipo_de_calle,Calle,Cruce,Comuna,Longitud,Latitud,Victima,Acusado
0,2016-0001,1,2016-01-01,2016,1,4,AVENIDA,PIEDRA BUENA AV.,1,8,-58.47533969,-34.68757022,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,1,GRAL PAZ,"PAZ, GRAL. AV.",1,9,-58.50877521,-34.66977709,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,7,AVENIDA,ENTRE RIOS AV.,0,1,-58.39040293,-34.63189362,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,0,AVENIDA,LARRAZABAL AV.,1,8,-58.46503904,-34.68092974,MOTO,None
4,2016-0005,1,2016-01-21,2016,1,5,AVENIDA,SAN JUAN AV.,1,1,-58.38718297,-34.62246630,MOTO,PASAJEROS


In [38]:
df_hechos_clean = df_hechos_clean.rename(columns={"Aaaa":"Año","Mm":"Mes","Hh":"Hora"})

In [39]:
df_hechos_clean.head()

,Id,N_victimas,Fecha,Año,Mes,Hora,Tipo_de_calle,Calle,Cruce,Comuna,Longitud,Latitud,Victima,Acusado
0,2016-0001,1,2016-01-01,2016,1,4,AVENIDA,PIEDRA BUENA AV.,1,8,-58.47533969,-34.68757022,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,1,GRAL PAZ,"PAZ, GRAL. AV.",1,9,-58.50877521,-34.66977709,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,7,AVENIDA,ENTRE RIOS AV.,0,1,-58.39040293,-34.63189362,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,0,AVENIDA,LARRAZABAL AV.,1,8,-58.46503904,-34.68092974,MOTO,None
4,2016-0005,1,2016-01-21,2016,1,5,AVENIDA,SAN JUAN AV.,1,1,-58.38718297,-34.62246630,MOTO,PASAJEROS


In [40]:
ConteoColumnas(df_hechos_clean)

Id
2021-0097    1
2016-0001    1
2016-0002    1
2016-0003    1
2016-0004    1
            ..
2016-0037    1
2016-0038    1
2016-0039    1
2016-0041    1
2016-0043    1
Name: count, Length: 695, dtype: int64
----------------------------------------------------------------------------------
N_victimas
1    675
2     19
3      1
Name: count, dtype: int64
----------------------------------------------------------------------------------
Fecha
2018-12-18    3
2018-12-22    3
2018-12-12    3
2018-04-27    3
2016-11-26    3
             ..
2017-11-28    1
2017-11-29    1
2017-11-30    1
2017-12-06    1
2017-11-07    1
Name: count, Length: 598, dtype: int64
----------------------------------------------------------------------------------
Año
2016    143
2018    143
2017    131
2019    103
2021     97
2020     78
Name: count, dtype: int64
----------------------------------------------------------------------------------
Mes
12    78
11    66
8     65
1     62
5     60
6     58
2     56
3     5

In [41]:
df_hechos_clean.to_parquet('../archivos/datos_limpios/hechos.parquet')

___

#########################################################################################################
___
**ETL Victimas**

In [565]:
df_victimas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 717 entries, 0 to 716
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ID_hecho             717 non-null    object        
 1   FECHA                717 non-null    datetime64[ns]
 2   AAAA                 717 non-null    int64         
 3   MM                   717 non-null    int64         
 4   DD                   717 non-null    int64         
 5   ROL                  717 non-null    object        
 6   VICTIMA              717 non-null    object        
 7   SEXO                 717 non-null    object        
 8   EDAD                 717 non-null    object        
 9   FECHA_FALLECIMIENTO  717 non-null    object        
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 56.1+ KB


In [566]:
df_victimas = df_victimas.replace('SD',None)

In [567]:
CalcularNulos(df_victimas)

el porcentaje de datos nulos en la columna  ID_hecho  es del  0.0 %
el porcentaje de datos nulos en la columna  FECHA  es del  0.0 %
el porcentaje de datos nulos en la columna  AAAA  es del  0.0 %
el porcentaje de datos nulos en la columna  MM  es del  0.0 %
el porcentaje de datos nulos en la columna  DD  es del  0.0 %
el porcentaje de datos nulos en la columna  ROL  es del  1.5 %
el porcentaje de datos nulos en la columna  VICTIMA  es del  1.3 %
el porcentaje de datos nulos en la columna  SEXO  es del  0.8 %
el porcentaje de datos nulos en la columna  EDAD  es del  7.4 %
el porcentaje de datos nulos en la columna  FECHA_FALLECIMIENTO  es del  9.5 %
-------------------------------------------------------------------------------------------


In [568]:
df_victimas.describe(include='all')

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
count,717,717,717.000000,717.000000,717.000000,706,708,711,664.0,649
unique,696,NaN,NaN,NaN,NaN,4,7,2,85.0,562
top,2017-0035,NaN,NaN,NaN,NaN,CONDUCTOR,MOTO,MASCULINO,30.0,2019-12-18 00:00:00
freq,3,NaN,NaN,NaN,NaN,330,303,545,28.0,3
mean,NaN,2018-09-09 01:36:24.100418304,2018.174338,6.688982,15.960948,NaN,NaN,NaN,NaN,NaN
min,NaN,2016-01-01 00:00:00,2016.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN
25%,NaN,2017-04-01 00:00:00,2017.000000,4.000000,9.000000,NaN,NaN,NaN,NaN,NaN
50%,NaN,2018-07-07 00:00:00,2018.000000,7.000000,16.000000,NaN,NaN,NaN,NaN,NaN
75%,NaN,2019-12-18 00:00:00,2019.000000,10.000000,23.000000,NaN,NaN,NaN,NaN,NaN
max,NaN,2021-12-30 00:00:00,2021.000000,12.000000,31.000000,NaN,NaN,NaN,NaN,NaN


In [569]:
df_victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18,None
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29,2016-02-01 00:00:00


rellenamos las columnas FECHA_FALLECIMIENTO con la columna FECHA *(ya que es una columna relativamente aproximada)*

In [570]:
df_victimas['FECHA_FALLECIMIENTO'] = df_victimas['FECHA_FALLECIMIENTO'].fillna(df_victimas['FECHA'])

eliminamos las siguientes columnas ya que estan visibles para el dia del hecho, la cual ya estan en la tabla hechos.parquet

In [571]:
df_victimas = df_victimas.drop(['FECHA','AAAA','MM','DD'],axis=1)

In [572]:
CalcularNulos(df_victimas)

el porcentaje de datos nulos en la columna  ID_hecho  es del  0.0 %
el porcentaje de datos nulos en la columna  ROL  es del  1.5 %
el porcentaje de datos nulos en la columna  VICTIMA  es del  1.3 %
el porcentaje de datos nulos en la columna  SEXO  es del  0.8 %
el porcentaje de datos nulos en la columna  EDAD  es del  7.4 %
el porcentaje de datos nulos en la columna  FECHA_FALLECIMIENTO  es del  0.0 %
-------------------------------------------------------------------------------------------


In [573]:
df_victimas.columns = [col.capitalize() for col in df_victimas.columns]

In [574]:
df_victimas[df_victimas['Victima'].isnull()]


,Id_hecho,Rol,Victima,Sexo,Edad,Fecha_fallecimiento
36,2016-0049,None,None,None,None,2016-04-17 00:00:00
77,2016-0101,None,None,MASCULINO,67,2016-08-07 00:00:00
89,2016-0115,None,None,MASCULINO,None,2016-09-02 00:00:00
93,2016-0119,PASAJERO_ACOMPAÑANTE,None,FEMENINO,None,2016-09-04 00:00:00
141,2016-0174,None,None,None,None,2016-12-27 00:00:00
167,2017-0029,None,None,MASCULINO,34,2017-03-07 00:00:00
208,2017-0074,None,None,MASCULINO,70,2017-06-04 00:00:00
221,2017-0089,None,None,MASCULINO,23,2017-07-13 00:00:00
280,2017-0155,None,None,MASCULINO,77,2017-12-12 00:00:00


In [575]:
df_hechos_clean[df_hechos_clean['Victima'].isnull()]

,Id,N_victimas,Año,Mes,Dia,Hora,Tipo_de_calle,Calle,Cruce,Comuna,Longitud,Latitud,Victima,Acusado
35,2016-0049,1,2016,4,17,0,AUTOPISTA,AUTOPISTA 1 SUR PRESIDENTE ARTURO FRONDIZI,,4,-58.37714647568196,-34.63657525428238,None,None
76,2016-0101,1,2016,8,7,19,GRAL PAZ,"PAZ, GRAL. AV.","CHAVEZ, JORGE",9,-58.52932872,-34.65014993,None,None
88,2016-0115,1,2016,9,2,11,CALLE,ARIAS,TRONADOR,12,-58.48935041,-34.54917673,None,CARGAS
92,2016-0119,1,2016,9,4,6,AVENIDA,"LARRALDE, CRISOLOGO AV.",TRIUNVIRATO AV.,12,-58.49659234,-34.56353635,None,None
139,2016-0174,1,2016,12,27,0,AUTOPISTA,AUTOPISTA 25 DE MAYO,,0,.,.,None,None
164,2017-0029,1,2017,3,7,2,CALLE,MOZART,TANDIL,9,-58.48501341,-34.64742455,None,AUTO
201,2017-0074,1,2017,6,4,12,CALLE,CASCO,,9,-58.52567073,-34.63787791,None,None
214,2017-0089,1,2017,7,13,12,CALLE,"LEGUIZAMON, MARTINIANO",,9,-58.51514709,-34.64787051,None,AUTO
269,2017-0155,1,2017,12,12,21,AVENIDA,RIVADAVIA AV.,,7,-58.45593915,-34.62656868,None,MOTO


miro que no hay formas de recuperar datos de las victimas de una tabla a otra, se deja sin efecto

+ extraigo el dia, sin las horas, ya que todas marcan 00:00:00 y no aportan nada al analisis

In [588]:
df_victimas['Fecha_fallecimiento'] = pd.to_datetime(df_victimas['Fecha_fallecimiento']).dt.date

por ultimo creo columnas para el dia, el mes y el año

In [592]:
df_victimas['Fecha_fallecimiento'] = pd.to_datetime(df_victimas['Fecha_fallecimiento'])

df_victimas['Año'] = df_victimas['Fecha_fallecimiento'].dt.year
df_victimas['Mes'] = df_victimas['Fecha_fallecimiento'].dt.month
df_victimas['Dia'] = df_victimas['Fecha_fallecimiento'].dt.day

In [593]:
df_victimas.to_parquet('../archivos/datos_limpios/victimas.parquet')